In [2]:
%load_ext autoreload
%autoreload 2

In [22]:
conv.outputShape

(5, 5, 1)

In [115]:
from NeuralNetwork import NeuralNetwork
from Layers import Dense, Conv, Reshape, sigmoid
from tests import *
from LayerTests.Convolutional import computeLocalGradient
from LayerTests.Convolutional import *
import matplotlib.pyplot as plt

conv = Conv((2,2),1,0,(2,2),sigmoid,inputShape=(5,5,1))
conv.allocateMemory(1)
ym1 = Tensor(np.random.randn(conv.batchSize,*conv.inputShape))
sigmaOut = Tensor((conv.batchSize,*conv.outputShape))

out1_ = 1
out2_ = 1


outInd1_ = np.zeros(conv.inputShape)
outInd2_ = np.zeros(conv.inputShape)
inInd1_ = np.zeros(conv.outputShape)
inInd2_ = np.zeros(conv.outputShape)
def forwardPropagate(globalIndex,ym1,outSize1,outSize2,filters,stride1,stride2,kernel1,kernel2,inSize1,inSize2,inSize3,padding,v,w,b):
    batch = globalIndex[0]
    filter = globalIndex[1]
    out1 = globalIndex[2]%outSize1
    out2 = globalIndex[2]//outSize1
    
    in1 = out1*stride1-padding
    in2 = out2*stride2-padding
    K1_LIMIT_INF = max(0,-in1)
    K2_LIMIT_INF = max(0,-in2)
    K1_LIMIT_SUP = min(kernel1,inSize1-in1)
    K2_LIMIT_SUP = min(kernel2,inSize2-in2)
    v[batch,out1,out2,filter] = b[filter,]
    for k1 in range(K1_LIMIT_INF, K1_LIMIT_SUP):
        for k2 in range(K2_LIMIT_INF, K2_LIMIT_SUP):
            for dim in range(inSize3):
                in1 = out1*stride1-padding+k1
                in2 = out2*stride2-padding+k2

                outInd1_[in1,in2] = out1+1
                outInd2_[in1,in2] = out2+1
                inInd1_[out1,out2] = in1+1
                inInd2_[out1,out2] = in2+1

                v[batch,out1,out2,filter] += \
                ym1[batch,in1,in2,dim] * \
                w[filter, k1, k2,dim]

outInd1 = np.zeros(conv.inputShape)
outInd2 = np.zeros(conv.inputShape)
inInd1 = np.zeros(conv.outputShape)
inInd2 = np.zeros(conv.outputShape)

def computeLocalGradient(globalIndex,sigmaOut,outSize1,outSize2,filters,inSize1,inSize2,inSize3,kernel1,kernel2,stride1,stride2,padding,sigmaIn,dphi,w):
    batch = globalIndex[0]
    in1 = globalIndex[1]//inSize2
    in2 = globalIndex[1]%inSize2
    in3 = globalIndex[2]
    K1_LIMIT_INF = max(0,in1-outSize1*stride1+padding+1)+in1%stride1
    K2_LIMIT_INF = max(0,in2-outSize2*stride2+padding+1)+in2%stride2
    K1_LIMIT_SUP = min(kernel1,in1+padding+1)
    K2_LIMIT_SUP = min(kernel2,in2+padding+1)
    sigmaIn[batch,in1,in2,in3] = 0
    for k1 in range(K1_LIMIT_INF,K1_LIMIT_SUP,stride1):
        for k2 in range(K2_LIMIT_INF,K2_LIMIT_SUP,stride2):
            for out3 in range(filters):
                out1 = (in1-k1+padding)//stride1
                out2 = (in2-k2+padding)//stride2
                if out1 == out1_ and out2 == out2_:
                    if in1 == 4 and in2 == 4:
                        print(in1,in2,out1,out2)
                        print(K1_LIMIT_INF,K1_LIMIT_SUP)
                    outInd1[in1,in2] = out1+1
                    outInd2[in1,in2] = out2+1
                    inInd1[out1,out2] = in1+1
                    inInd2[out1,out2] = in2+1
                sigmaIn[batch,in1,in2,in3] += w[out3,k1,k2,in3] \
                    *sigmaOut[batch,out1,out2,out3] \
                        *dphi[batch,out1,out2,out3]

output = runCPUKernel(forwardPropagate,conv.GPUForwardPropagate,(ym1,),ind=[0,0,out1_*conv.outputShape[1]+out2_])
ym1, v, w, b = output[0],output[-3],output[-2],output[-1] 
#print(v.accessed.squeeze())
#print(ym1.accessed.squeeze())

output = runCPUKernel(computeLocalGradient,conv.computeLocalGradient,(sigmaOut,))
#output[0].array[np.logical_not(output[0].accessed)] = 0
#print(output[0].array.squeeze())



4 4 1 1
1 2


In [116]:
print(outInd1_.squeeze())
print(outInd1.squeeze())

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 2. 2. 0.]
 [0. 0. 2. 2. 0.]
 [0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 2. 2. 2.]
 [0. 0. 2. 2. 2.]
 [0. 0. 2. 2. 2.]]


In [109]:
print(outInd2_.squeeze())
print(outInd2.squeeze())

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 2. 2. 0.]
 [0. 0. 2. 2. 0.]
 [0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 0.]
 [0. 0. 0. 0. 0.]]


In [88]:
print(inInd1_.squeeze())
print(inInd1.squeeze())

[[0. 0.]
 [0. 4.]]
[[0. 0.]
 [0. 5.]]


In [89]:
print(inInd2_.squeeze())
print(inInd2.squeeze())

[[0. 0.]
 [0. 4.]]
[[0. 0.]
 [0. 5.]]


In [17]:
from Tensor import Tensor
a = Tensor((10,))
list = [a,1]
a = Tensor((1,))
list[0]

[0.00778198 0.00778198 0.00778198 0.00778198 0.00778198 0.00778198
 0.00778198 0.00778198 0.00778198 0.00778198]

In [8]:
from settings import convolutionalcl
convolutionalcl.forwardPropagate

15

TypeError: getLoss() missing 1 required positional argument: 'Y'

In [7]:
model.e

[[ 0.5028373  -0.43265942 -0.74772906]]

In [1]:
from NeuralNetwork import NeuralNetwork
from Layers import Dense, Conv, Reshape, sigmoid
from tests import *
from LayerTests.Convolutional import *
import matplotlib.pyplot as plt
from tests import gradientTest
model = NeuralNetwork([Conv((5,5),6,2,(2,2),sigmoid,inputShape=(10,10,1)),
                        #Conv((3,3),6,0,(1,1),sigmoid),
                        Conv((2,2),6,1,(2,2),sigmoid), # GRADIENT LOCALS NOT WORKING PROPPERLY
                        Reshape(-1), 
                        Dense(3,sigmoid)])

gradientTest(model)

FAILED Layer 0 dw r2= 0.13, m= -1.2, b= 0.0,
FAILED Layer 0 db r2= 0.29, m= -0.56, b= -0.01,
FAILED Layer 1 dw r2= 0.2, m= -1.04, b= 0.0,
FAILED Layer 1 db r2= 0.45, m= -1.3, b= 0.0,
PASSED Layer 2 dw r2= 1.0, m= 3.01, b= 0.0,
PASSED Layer 2 db r2= 1.0, m= 3.0, b= 0.0,
PASSED Layer 4 dw r2= 1.0, m= 3.01, b= 0.0,
PASSED Layer 4 db r2= 1.0, m= 3.01, b= 0.0,


([[-0.014051198959350586,
   0.013710260391235352,
   0.0011110305786132812,
   -0.01994172732035282,
   0.009640057881673547,
   -0.00866611798604311,
   0.010752677917480469,
   -0.03388086954752567,
   -0.01231074333190918,
   0.011648337046305524,
   0.0009334087371826172,
   -0.0042569637298583984,
   0.02778569857279478,
   0.0014845530192059142,
   0.012840827306112024,
   0.017500321070353375,
   -0.018607378005981445,
   -0.00035007794698060923,
   0.010548830032348633,
   -0.002994537353515625,
   0.02711176872253418,
   -0.00769654909769657,
   0.0037399927775066955,
   0.01196146011352539,
   0.014330546061198102,
   0.0006790955861413206,
   0.004664262135823938,
   0.004575649897257672,
   0.00038743019104003906,
   0.007474422454833984,
   -0.006792942682901648,
   0.0027974446614587034,
   0.000676711400350305,
   -0.001615285873413086,
   0.0028705596923828125,
   0.014040867487589703,
   0.0037197271982830626,
   0.011766354242960797,
   -0.0055297215779620545,
   0.0

In [3]:
from NeuralNetwork import NeuralNetwork
from Layers import Dense, Conv, Reshape, sigmoid
from tests import *
from LayerTests.Convolutional import *
import matplotlib.pyplot as plt
from tests import gradientTest
model = NeuralNetwork([Conv((5,5),6,2,(2,2),sigmoid,inputShape=(10,10,1)),
                        #Conv((3,3),6,0,(1,1),sigmoid),
                        Conv((2,2),6,0,(2,2),sigmoid),
                        Reshape(-1), 
                        Dense(3,sigmoid)])
model.allocateMemory(1)

0.54297835
PASSED forwardPropagate
PASSED sigmoidTest
PASSED computedb
PASSED computeGradients
PASSED computeLocalGradient
1.0
1.0
1.0
1.0
1.0
1.0
FAILED learningRule
Param Index 6 with name "w" was off by 9.701629638671875
Param Index 7 with name "b" was off by 8.960834503173828

-42.398045


In [16]:
from NeuralNetwork import NeuralNetwork
from Layers import Dense, Conv, Reshape, sigmoid
from tests import *
from LayerTests.Convolutional import *
import matplotlib.pyplot as plt
from tests import gradientTest
model = NeuralNetwork([Conv((5,5),6,2,(2,2),sigmoid,inputShape=(10,10,1)),
                        #Conv((3,3),6,0,(1,1),sigmoid),
                        Conv((2,2),6,0,(2,2),sigmoid),
                        Reshape(-1), 
                        Dense(3,sigmoid)])

dw,db,plotter = gradientTest(model)
model

FAILED Layer 0 dw r2= 0.57, m= 3.04, b= 0.0,
FAILED Layer 0 db r2= 0.89, m= 5.5, b= 0.0,
PASSED Layer 1 dw r2= 1.0, m= 2.98, b= -0.0,
PASSED Layer 1 db r2= 1.0, m= 2.99, b= -0.0,
PASSED Layer 3 dw r2= 1.0, m= 2.9, b= 0.0,
PASSED Layer 3 db r2= 1.0, m= 2.9, b= -0.0,


		 MODEL SUMMARY: 
Input: 		 (10, 10, 1)
Conv Layer: 		(5, 5, 6)
Conv Layer: 		(2, 2, 6)
Reshape Layer: 		(24,)
Dense Layer: 		(3,)

In [3]:
from tests import *
from LayerTests.Convolutional import *
batchSize = 10
conv = Conv((5,5),3,1,(2,2),sigmoid, inputShape=(10,10,1))
conv.allocateMemory(batchSize)
ym1 = Tensor(np.random.randn(batchSize,*conv.inputShape))
sigmaOut = Tensor(np.random.randn(batchSize,*conv.outputShape))



PASSED forwardPropagate
PASSED sigmoidTest
PASSED computedb
PASSED computeGradients
PASSED computeLocalGradient
FAILED learningRule
Param Index 6 with name "w" was off by 4.580564975738525
Param Index 7 with name "b" was off by 0.7990346550941467



In [9]:
sigmaOut.shape

(10, 8, 8, 2)

In [8]:
sigmaOut.get()[:,:,1]

array([[[0., 0.],
        [1., 1.],
        [2., 2.],
        [3., 3.],
        [4., 4.],
        [5., 5.],
        [6., 6.],
        [7., 7.]],

       [[7., 7.],
        [8., 1.],
        [2., 2.],
        [3., 3.],
        [4., 4.],
        [5., 5.],
        [6., 6.],
        [7., 7.]],

       [[7., 7.],
        [8., 1.],
        [2., 2.],
        [3., 3.],
        [4., 4.],
        [5., 5.],
        [6., 6.],
        [7., 7.]],

       [[7., 7.],
        [8., 1.],
        [2., 2.],
        [3., 3.],
        [4., 4.],
        [5., 5.],
        [6., 6.],
        [7., 7.]],

       [[7., 7.],
        [8., 1.],
        [2., 2.],
        [3., 3.],
        [4., 4.],
        [5., 5.],
        [6., 6.],
        [7., 7.]],

       [[7., 7.],
        [8., 1.],
        [2., 2.],
        [3., 3.],
        [4., 4.],
        [5., 5.],
        [6., 6.],
        [7., 7.]],

       [[7., 7.],
        [8., 1.],
        [2., 2.],
        [3., 3.],
        [4., 4.],
        [5., 5.],
        [6., 6.]

In [2]:
print(out2['not accesed ym1'][0,:,:,0])

NameError: name 'out2' is not defined